# Practice Session 08: Data streams


Author: <font color="blue">Marc Pérez Pratdesaba</font>

E-mail: <font color="blue">marc.perez12@estudiant.upf.edu</font>

Date: <font color="blue">05/12/2022</font>

In this practice we will take a large corpus of documents and compute some statistics using data streams methods.

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re

# 0. Dataset and how to iterate

In [2]:
INPUT_FILE = "movie_lines.tsv.gz"

In [3]:
# Producer in Python that reads a filename by words

def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 3 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z]{2,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(text):
                          
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [4]:
# Iterate through the file

for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 1/10000 of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word)) 

Current word 'now'
Current word 'na'
Current word 'ought'
Current word 'earth'
Current word 'lot'
Current word 'as'
Current word 'charlie'
Current word 'jesse'
Current word 'gang'
Current word 'teddy'
Current word 'could'
- Read 100000/300000 words so far
Current word 'if'
Current word 'who'
Current word 'owns'
Current word 'used'
Current word 'over'
Current word 'the'
- Read 200000/300000 words so far
Current word 'what'
Current word 'she'
Current word 'not'
Current word 'tell'
Current word 'thirsty'
Current word 'understand'
Current word 'coming'
Current word 'die'
Current word 'you'
- Read 300000/300000 words so far


# 1. Determine approximately the top-5 words

In [5]:
def add_to_reservoir(reservoir, item, max_reservoir_size):

    assert(len(reservoir) <= max_reservoir_size) # We assert that the lenght is not greater than max_reservoir_size
    reservoir.pop(random.randrange(len(reservoir))) # If reservoir is max_size, we pop one item randomly
    reservoir.append(item) # We append the new item in the reservoir
    return reservoir

In [6]:
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []

    words_read = 0

    for word in read_by_words(filename, max_words=max_words, report_every=report_every):

            if len(reservoir) < reservoir_size: # We append the first nth items
                reservoir.append(word)
            else:
                if random.random() < (1 - reservoir_size/words_read): # If it falls within the probability range, we call add_to_reservoir
                    reservoir = add_to_reservoir(reservoir, word, reservoir_size)
            words_read += 1

    return (words_read, reservoir)

In [7]:
reservoir_size = 1000
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=1000000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000028
Number of items sampled : 1000


In [8]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

45 you
32 the
32 it
25 to
19 and
14 do
13 was
13 this
12 on
12 me


In [9]:
print(f"Word    Relative Frequency\n")
for absolute_frequency, word in most_frequent_items:
    print(f" {word}         {round((100*absolute_frequency/len(reservoir)), 3)}% ")


Word    Relative Frequency

 you         4.5% 
 the         3.2% 
 it         3.2% 
 to         2.5% 
 and         1.9% 
 do         1.4% 
 was         1.3% 
 this         1.3% 
 on         1.2% 
 me         1.2% 


In [10]:
reservoir_sizes = [50, 500, 3000]

for size in reservoir_sizes:
    (items_seen, reservoir) = reservoir_sampling(INPUT_FILE, size, max_words=1000000, report_every=100000)
    
    freq = {}
    for item in reservoir:
        freq[item] = reservoir.count(item)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
    
    print(f"\n For reservoir size {size}:")
    for absolute_frequency, word in most_frequent_items:
        print(f" {word}   {absolute_frequency}   {round((100*absolute_frequency/len(reservoir)), 3)}%")
    print("\n")

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far

 For reservoir size 50:
 my   3   6.0%
 it   3   6.0%
 your   2   4.0%
 you   2   4.0%
 think   2   4.0%
 something   2   4.0%
 give   2   4.0%
 case   2   4.0%
 wo   1   2.0%
 what   1   2.0%


- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far

 For reservoir size 500:
 it   23   4.6%
 you   18   3.6%
 the   17   3.4%
 to   13   2.6%
 and   12 

In [11]:
recommended_size = 3000

(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, recommended_size, max_words=3000000, report_every=100000) #Very large number for limit which indicates that there is none
    
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:3]

print("\n")
for absolute_frequency, word in most_frequent_items:
    print(f" {word}   {absolute_frequency}   {round((100*absolute_frequency/len(reservoir)), 3)}%")

- Read 100000/3000000 words so far
- Read 200000/3000000 words so far
- Read 300000/3000000 words so far
- Read 400000/3000000 words so far
- Read 500000/3000000 words so far
- Read 600000/3000000 words so far
- Read 700000/3000000 words so far
- Read 800000/3000000 words so far
- Read 900000/3000000 words so far
- Read 1000000/3000000 words so far
- Read 1100000/3000000 words so far
- Read 1200000/3000000 words so far
- Read 1300000/3000000 words so far
- Read 1400000/3000000 words so far
- Read 1500000/3000000 words so far
- Read 1600000/3000000 words so far
- Read 1700000/3000000 words so far
- Read 1800000/3000000 words so far
- Read 1900000/3000000 words so far
- Read 2000000/3000000 words so far
- Read 2100000/3000000 words so far
- Read 2200000/3000000 words so far
- Read 2300000/3000000 words so far
- Read 2400000/3000000 words so far
- Read 2500000/3000000 words so far
- Read 2600000/3000000 words so far
- Read 2700000/3000000 words so far
- Read 2800000/3000000 words so far
-

The size I would recommend using is **3000**, which it relatively gets always the same top words at each computation of the algorithm. We can observe that the top 3 words are "you", "the", and "to", which makes sense since they are very frequently used words when writing or speaking english.

# 2. Determine approximately the distinct number of words

In [12]:
def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [13]:
def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

In [14]:
number_of_passes = 20
estimates = []
high_max_words = 300000000
for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    hash_func = random_hash_function()
    max_R = 0
    for u in read_by_words(INPUT_FILE, max_words=high_max_words, report_every=100000):
        h_u = hash_func(u)
        r_u = count_trailing_zeroes(h_u)
        max_R = max(r_u, max_R)
        
    estimate = 2**max_R
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))

- Read 100000/300000000 words so far
- Read 200000/300000000 words so far
- Read 300000/300000000 words so far
- Read 400000/300000000 words so far
- Read 500000/300000000 words so far
- Read 600000/300000000 words so far
- Read 700000/300000000 words so far
- Read 800000/300000000 words so far
- Read 900000/300000000 words so far
- Read 1000000/300000000 words so far
- Read 1100000/300000000 words so far
- Read 1200000/300000000 words so far
- Read 1300000/300000000 words so far
- Read 1400000/300000000 words so far
- Read 1500000/300000000 words so far
- Read 1600000/300000000 words so far
- Read 1700000/300000000 words so far
- Read 1800000/300000000 words so far
- Read 1900000/300000000 words so far
- Read 2000000/300000000 words so far
- Read 2100000/300000000 words so far
- Read 2200000/300000000 words so far
- Read 2300000/300000000 words so far
- Read 2400000/300000000 words so far
- Read 2500000/300000000 words so far
- Read 2600000/300000000 words so far
- Read 2700000/300000

- Read 800000/300000000 words so far
- Read 900000/300000000 words so far
- Read 1000000/300000000 words so far
- Read 1100000/300000000 words so far
- Read 1200000/300000000 words so far
- Read 1300000/300000000 words so far
- Read 1400000/300000000 words so far
- Read 1500000/300000000 words so far
- Read 1600000/300000000 words so far
- Read 1700000/300000000 words so far
- Read 1800000/300000000 words so far
- Read 1900000/300000000 words so far
- Read 2000000/300000000 words so far
- Read 2100000/300000000 words so far
- Read 2200000/300000000 words so far
- Read 2300000/300000000 words so far
- Read 2400000/300000000 words so far
- Read 2500000/300000000 words so far
- Read 2600000/300000000 words so far
- Read 2700000/300000000 words so far
- Read 2800000/300000000 words so far
- Read 2900000/300000000 words so far
Estimate on pass 8: 32768 distinct words
- Read 100000/300000000 words so far
- Read 200000/300000000 words so far
- Read 300000/300000000 words so far
- Read 400000/

- Read 1500000/300000000 words so far
- Read 1600000/300000000 words so far
- Read 1700000/300000000 words so far
- Read 1800000/300000000 words so far
- Read 1900000/300000000 words so far
- Read 2000000/300000000 words so far
- Read 2100000/300000000 words so far
- Read 2200000/300000000 words so far
- Read 2300000/300000000 words so far
- Read 2400000/300000000 words so far
- Read 2500000/300000000 words so far
- Read 2600000/300000000 words so far
- Read 2700000/300000000 words so far
- Read 2800000/300000000 words so far
- Read 2900000/300000000 words so far
Estimate on pass 15: 32768 distinct words
- Read 100000/300000000 words so far
- Read 200000/300000000 words so far
- Read 300000/300000000 words so far
- Read 400000/300000000 words so far
- Read 500000/300000000 words so far
- Read 600000/300000000 words so far
- Read 700000/300000000 words so far
- Read 800000/300000000 words so far
- Read 900000/300000000 words so far
- Read 1000000/300000000 words so far
- Read 1100000/30

In [15]:
print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 117964.8
* Median  of estimates: 32768.0


For number of passes = 10:


    
Run_1_avg = **114688.0**
Run_1_med = **65536.0**



Run_2_avg = **141721.6**
Run_2_med = **24576.0**



Run_3_avg = **308019.2**
Run_3_med = **98304.0**



For number of passes = 20:


Run_1_avg = **109363.2**
Run_1_med = **49152.0**



Run_2_avg = **134348.8**
Run_2_med = **65536.0**



Run_3_avg = **117964.8**
Run_3_med = **32768.0**



We can notice that for the runnings of num_passes = 20, we get more stable and appropiate results. Furthermore, we can notice that the median is relatively separated from the median on the majority of the cases. This can be due to outliers (which bias the average), or the results being very different in each case.


We may conclude that it is due to the second case because our algorithm is based on a random hash function, and everytime we will get different results.